# Use Snowpark_OpenAI Notebook template

## Import Pandas for reading a csv

In [1]:
!pip install -q pandas

ERROR: snowflake-ml-python 1.0.10 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.2.2 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement snowflake-snowpark-python<2,>=1.5.1, but you'll have snowflake-snowpark-python 1.5.0 which is incompatible.
ERROR: snowflake-connector-python 3.7.1 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
ERROR: botocore 1.34.51 has requirement urllib3<1.27,>=1.25.4; python_version < "3.10", but you'll have urllib3 2.1.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Import Snowpark Pandas for dumping csv to Snowflake table

In [2]:
!pip install "snowflake-snowpark-python[pandas]"

     |████████████████████████████████| 419kB 5.3MB/s eta 0:00:01
     |████████████████████████████████| 71kB 22.2MB/s eta 0:00:01
     |████████████████████████████████| 2.6MB 58.8MB/s eta 0:00:01
     |████████████████████████████████| 901kB 79.3MB/s eta 0:00:01
     |████████████████████████████████| 747kB 73.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 20.1MB/s eta 0:00:01
     |████████████████████████████████| 153kB 67.1MB/s eta 0:00:01
     |████████████████████████████████| 163kB 76.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 71.2MB/s eta 0:00:01
     |████████████████████████████████| 4.6MB 78.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 19.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl
     |████████████████████████████████| 112kB 81.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 

In [3]:
from snowflake.snowpark.session import Session
import pandas as pd
import subprocess

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

## Code to establish connection and read and dump csv as a snowflake table (handling null values before loading)

In [10]:
#Import all snowflake connection details from template variables.

db_user = 'SPECTRA_DEMO_USER'
db_password = 'Spectra@2023'
db_account = 'fya62509'
db_database =  os.getenv('sf_db')
db_role = os.getenv('sf_role')
db_warehouse = 'FOSFOR_FDC'
db_schema = 'PUBLIC'

In [11]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [12]:
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

df =  pd.read_csv("/notebooks/notebooks/customer_data.csv")
df.fillna(0, inplace=True)
df_model=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_model.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CC_CUSTOMER_DATA")

In [13]:
df =  pd.read_csv("/notebooks/notebooks/payment_data.csv")
df.head()

,id,OVD_t1,OVD_t2,OVD_t3,OVD_sum,pay_normal,prod_code,prod_limit,update_date,new_balance,highest_balance,report_date
0,58987402,0,0,0,0,1,10,16500.0,04/12/2016,0.0,NaN,NaN
1,58995151,0,0,0,0,1,5,NaN,04/12/2016,588720.0,491100.0,NaN
2,58997200,0,0,0,0,2,5,NaN,04/12/2016,840000.0,700500.0,22/04/2016
3,54988608,0,0,0,0,3,10,37400.0,03/12/2016,8425.2,7520.0,25/04/2016
4,54987763,0,0,0,0,2,10,NaN,03/12/2016,15147.6,NaN,26/04/2016


In [14]:
df.isnull().sum()

id                    0
OVD_t1                0
OVD_t2                0
OVD_t3                0
OVD_sum               0
pay_normal            0
prod_code             0
prod_limit         6118
update_date          26
new_balance           0
highest_balance     409
report_date        1114
dtype: int64

In [15]:
df[['prod_limit','highest_balance']] = df[['prod_limit','highest_balance']].fillna(0)
df[['update_date','report_date']] = df[['prod_limit','highest_balance']].fillna('31/12/9999')

In [16]:
df.isnull().sum()

id                 0
OVD_t1             0
OVD_t2             0
OVD_t3             0
OVD_sum            0
pay_normal         0
prod_code          0
prod_limit         0
update_date        0
new_balance        0
highest_balance    0
report_date        0
dtype: int64

## Code to establish connection and read and dump csv as a snowflake table (handling null values before loading)

In [17]:
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

df =  pd.read_csv("/notebooks/notebooks/payment_data.csv")
df[['prod_limit','highest_balance']] = df[['prod_limit','highest_balance']].fillna(0)
df[['update_date','report_date']] = df[['update_date','report_date']].fillna('31/12/9999')
df_model=session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_model.write.mode("overwrite").save_as_table("FDC_DATA_MANISH.PUBLIC.CC_PAYMENT_DATA")